In [1]:
import keras
import tensorflow
from keras import Layer
from keras.layers import Dense

In [2]:
def model_build(hp):
    model = keras.Sequential()
    
    model.add(Dense(units = hp.Int('units', min_value = 32, max_value = 512, step = 32), activation = 'relu'))
    
    model.add(Dense(10, activation = {'softmax'}))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model